<a href="https://colab.research.google.com/github/rashidsaidhamad/rain-website/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()


Saving Modified_SQL_Dataset.csv to Modified_SQL_Dataset.csv


In [2]:
import pandas as pd

# Load and clean column names
df = pd.read_csv("Modified_SQL_Dataset.csv")
df.columns = df.columns.str.strip()  # Remove extra spaces in column names

# Check first few rows
print(df.columns)
df.head()


Index(['Query', 'Label'], dtype='object')


,Query,Label
0,""" or pg_sleep ( __TIME__ ) --",1
1,create user name identified by pass123 tempora...,1
2,AND 1 = utl_inaddr.get_host_address ( ...,1
3,select * from users where id = '1' or @ @1 ...,1
4,"select * from users where id = 1 or 1#"" ( ...",1


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

X = df['Query']
y = df['Label']

# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
X_vec = vectorizer.fit_transform(X)

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, random_state=42)


In [4]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      0.70      0.82      3893
           1       0.66      1.00      0.80      2291

    accuracy                           0.81      6184
   macro avg       0.83      0.85      0.81      6184
weighted avg       0.87      0.81      0.81      6184



In [5]:
import joblib

# Save the trained model
joblib.dump(rf, 'random_forest_model.pkl')


['random_forest_model.pkl']

In [6]:
from google.colab import files
files.download('random_forest_model.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
!pip install --upgrade transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 125.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.2
    Uninstalling transformers-4.52.2:
      Successfully uninstalled transformers-4.52.2


In [9]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['Label'])


In [10]:
from transformers import BertTokenizer
from torch.utils.data import Dataset
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class SQLDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=128)
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    df['Query'],
    df['label_encoded'],
    test_size=0.2,
    stratify=df['label_encoded'],
    random_state=42
)

train_dataset = SQLDataset(X_train.tolist(), y_train.tolist())
test_dataset = SQLDataset(X_test.tolist(), y_test.tolist())


In [16]:
import transformers
print(transformers.__version__)


4.52.4


In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    save_strategy="epoch",  # <-- Added to match eval_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
)


In [13]:
from huggingface_hub import notebook_login
notebook_login()

In [14]:
from transformers import BertForSequenceClassification, Trainer
from huggingface_hub import notebook_login

# Login first (only needed once per session)
notebook_login()

# Then proceed with your code
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(le.classes_))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()
results = trainer.evaluate()
print("Evaluation results:", results)
trainer.save_model("./best_bert_model")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shdrashid95 (shdrashid95-state-university-of-zanzibar) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.012600,0.010174
2,0.006000,0.007424
3,0.000400,0.009239


Evaluation results: {'eval_loss': 0.007424423471093178, 'eval_runtime': 40.9904, 'eval_samples_per_second': 150.864, 'eval_steps_per_second': 2.366, 'epoch': 3.0}


In [15]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {"accuracy": acc, "f1": f1}


In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)


In [17]:
from transformers import pipeline

# Load your saved model
classifier = pipeline(
    "text-classification",
    model="./best_bert_model",  # or your model path
    tokenizer="bert-base-uncased"
)

# Test with user input
user_input = "This is a sample text to classify"
prediction = classifier(user_input)

print(f"Input: {user_input}")
print(f"Predicted class: {prediction[0]['label']}")
print(f"Confidence: {prediction[0]['score']:.4f}")

Device set to use cuda:0


Input: This is a sample text to classify
Predicted class: LABEL_1
Confidence: 0.9874


In [18]:
!zip -r best_bert_model.zip best_bert_model/



  adding: best_bert_model/ (stored 0%)
  adding: best_bert_model/model.safetensors (deflated 7%)
  adding: best_bert_model/training_args.bin (deflated 51%)
  adding: best_bert_model/config.json (deflated 49%)


In [19]:
from google.colab import files
files.download("best_bert_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>